In [ ]:
# IMPORT BIASANALYZER COMPONENTS
from biasanalyzer.api import BIAS
from CohortViewer import *
from CohortDefinition import (ConditionOccurrence, VisitOccurrence, DateEvent, Demographics, CohortCriteria, AND, OR, BEFORE, NOT,)

# CREATE INSTANCE OF BIAS & CONNECT TO DATA SOURCE
bias = BIAS()
bias.set_config("../config_duckdb_real.yaml")
try:
    bias.set_root_omop()
except:
    bias.cleanup()
    raise  # continue to raise the error

In [ ]:
# DEFINE THE STUDY COHORT

# type 2 diabetes mellitus (T2D) first instance
t2d_first = ConditionOccurrence(event_concept_id=201826, event_instance=1)
# chronic kidney disease (CKD) first instance
ckd_first = ConditionOccurrence(event_concept_id=46271022, event_instance=1)
# CKD offset before diagnosis
ckd_prior = ConditionOccurrence(event_concept_id=46271022, event_instance=1, offset=0)
# CKD offset after diagnosis
ckd_post = ConditionOccurrence(event_concept_id=46271022, event_instance=1, offset=730)

# T2D diagnosis occurs before CKD
group1 = BEFORE(t2d_first, ckd_first)
# collect data from CKD first diagnosis to 730 days (2 years) after first diagnosis
group2 = AND(ckd_prior, ckd_post)

rule = AND(group1, group2) # bring it all together into a single rule

# create the study cohort criteria as YAML
cohort1_def = CohortCriteria(
    temporal_blocks=[rule],
)
print(cohort1_def.to_yaml())

# DEFINE THE BASELINE COHORT

# all T2D patients
t2d = ConditionOccurrence(event_concept_id=201826)

# create the baseline criteria as YAML
cohort2_def = CohortCriteria(
    temporal_blocks=[t2d],
)
print(cohort2_def.to_yaml())

In [ ]:
# CREATE THE STUDY COHORT
cohort1 = bias.create_cohort(
    'Study Cohort: T2DM & CKD',
    ('Patients diagnosed with type 2 diabetes mellitus (T2D) prior to a chronic kidney disease (CKD) diagnosis, '
    'with an observation period of 2 years after initial CKD diagnosis.'),
    cohort1_def,
    'system'
)
# print(f'cohort1 id = {cohort1.cohort_id}')

In [ ]:
# VIEW THE STUDY COHORT
view1 = CohortViewer(cohort1=cohort1,
                     cohort1_shortname="T2D & CKD")
view1

In [ ]:
# DISCONNECT FROM THE DATABASE
bias.cleanup()